### Goal of this program: To take inputs (stats from tennis players) and output predicted winning percentage

### Steps:
###### 1) Scrape data from tennis database (tennisabstract.com for now)
###### 2) input data into model
###### 3) Model ouputs predicted winning percentage (and maybe predicted outcome against opponent in the future)

In [1]:
import numpy as np
import random
from random import randint
from sklearn.preprocessing import MinMaxScaler
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import *
from keras.models import Model
from keras.applications import imagenet_utils
from keras.preprocessing import image
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
import itertools
# from keras.utils.vis_utils import plot_model
%matplotlib inline

import os
import json
# import nbimporter


In [2]:
# from serve_scraper import data

# import DataTransfer

In [3]:
with open("data.json", "r") as file:
    data = json.load(file)
print(data)

{'Giovanni Mpetshi Perricard': ['No data?', 64.9, 15.53, 14.84, {'singles': [14.68, 77.0, -79.3, 66.22, 10], 'doubles': [14.06, -6.0, -37.23, 36.84, 3]}], 'Alexander Zverev': [126.79, 71.0, 16.15, 14.86, {'singles': [15.43, 298.0, 169.13, 76.74, 12], 'doubles': [14.29, 9.0, -12.24, 50.0, 4]}], 'Hubert Hurkacz': [126.79, 63.4, 15.82, 14.22, {'singles': [15.37, 103.0, 44.05, 67.27, 6], 'doubles': [13.94, -4.0, -7.47, 40.0, 1]}], 'Matteo Berrettini': [130.52, 68.2, 15.71, 15.44, {'singles': [15.2, 111.0, 62.5, 72.0, 10], 'doubles': [14.49, 4.0, 2.31, 100.0, 1]}], 'Jannik Sinner': [121.19, 61.4, 16.39, 15.44, {'singles': [15.51, 431.0, 305.03, 92.5, 29], 'doubles': [14.47, 18.0, 1.84, 55.56, 2]}], 'Ben Shelton': [137.97, 68.8, 15.77, 14.64, {'singles': [15.35, 76.0, 2.01, 62.12, 5], 'doubles': [14.14, -21.0, -53.49, 43.33, 3]}], 'Taylor Fritz': [123.06, 62.4, 16.12, 15.05, {'singles': [15.41, 199.0, 86.07, 69.33, 8], 'doubles': [14.19, 34.0, 1.31, 65.0, 3]}], 'Brandon Nakashima': [118.71, 

### Setting all the training/valid sets up 

In [4]:
# Data dependent on match type (singles and doubles) will be put in corresponding lists. These corresponding lists
# will all be put together in another list. This list is then paired with another list in a dictionary. For example,
# singles training sample data will be put in their corresponding lists. These lists will be put in a singles training
# sample list, which countains all the lists with the singles training sample data. This singles training list will 
# then be paired with a doubles training sample list in a dictionary. 

In [5]:
adv_training_sets = dict()
scaled_adv_training_sets = dict()


valid_sets = dict()
scaled_valid_sets = dict()

singlesUtr_train_samples = []
doublesUtr_train_samples = []
servePercentage_train_samples = []

scaled_singlesUtr_train_samples = []
scaled_doublesUtr_train_samples = []
scaled_servePercentage_train_samples = []

basic_training_sets = []
basic_training_sets.append(singlesUtr_train_samples)
basic_training_sets.append(doublesUtr_train_samples) 
basic_training_sets.append(servePercentage_train_samples)

scaled_basic_training_sets = []
scaled_basic_training_sets.append(scaled_singlesUtr_train_samples) 
scaled_basic_training_sets.append(scaled_doublesUtr_train_samples) 
scaled_basic_training_sets.append(scaled_servePercentage_train_samples)

singlesGameDiff_train_samples = []
singlesWeightedGameDiff_train_samples = []
singlesWinPercentage_train_samples = []
singlesOpponentUtr_train_samples = []
singlesLongestWinStreak_train_samples = []

scaled_singlesGameDiff_train_samples = []
scaled_singlesWeightedGameDiff_train_samples = []
scaled_singlesWinPercentage_train_samples = []
scaled_singlesOpponentUtr_train_samples = []
scaled_singlesLongestWinStreak_train_samples = []

adv_training_sets["singles"] = [] 
adv_training_sets["singles"].append(singlesGameDiff_train_samples) 
adv_training_sets["singles"].append(singlesWeightedGameDiff_train_samples) 
adv_training_sets["singles"].append(singlesWinPercentage_train_samples) 
adv_training_sets["singles"].append(singlesOpponentUtr_train_samples) 
adv_training_sets["singles"].append(singlesLongestWinStreak_train_samples)

scaled_adv_training_sets["singles"] = [] 
scaled_adv_training_sets["singles"].append(scaled_singlesGameDiff_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesWeightedGameDiff_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesWinPercentage_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesOpponentUtr_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesLongestWinStreak_train_samples)


# print(scaled_adv_training_sets)

'''
for i in range(5):   # Adding scaled values to all advanced stat training sets
    # list_ = adv_training_sets[match_type][i]
    scaled_list = scaled_adv_training_sets['singles'][i]
    scaled_adv_training_sets['singles'][i].extend([0, i])
    # print(f"adv_training_set scaled list: {type(scaled_adv_training_sets[dict_string][i])}")

print(scaled_adv_training_sets)

print(type(scaled_singlesGameDiff_train_samples))
'''

# print(singles_train_lists)

doublesGameDiff_train_samples = []
doublesWeightedGameDiff_train_samples = []
doublesWinPercentage_train_samples = []
doublesOpponentUtr_train_samples = []
doublesLongestWinStreak_train_samples = []

scaled_doublesGameDiff_train_samples = []
scaled_doublesWeightedGameDiff_train_samples = []
scaled_doublesWinPercentage_train_samples = []
scaled_doublesOpponentUtr_train_samples = []
scaled_doublesLongestWinStreak_train_samples = []

'''
adv_training_sets["doubles"] = [doublesGameDiff_train_samples, doublesWeightedGameDiff_train_samples, doublesWinPercentage_train_samples, doublesOpponentUtr_train_samples, doublesLongestWinStreak_train_samples]
scaled_adv_training_sets["doubles"] = [scaled_doublesGameDiff_train_samples, scaled_doublesWeightedGameDiff_train_samples, scaled_doublesWinPercentage_train_samples, scaled_doublesOpponentUtr_train_samples, scaled_doublesLongestWinStreak_train_samples]
'''

adv_training_sets["doubles"] = [] 
adv_training_sets["doubles"].append(doublesGameDiff_train_samples) 
adv_training_sets["doubles"].append(doublesWeightedGameDiff_train_samples) 
adv_training_sets["doubles"].append(doublesWinPercentage_train_samples) 
adv_training_sets["doubles"].append(doublesOpponentUtr_train_samples) 
adv_training_sets["doubles"].append(doublesLongestWinStreak_train_samples)
                                    
scaled_adv_training_sets["doubles"] = [] 
scaled_adv_training_sets["doubles"].append(scaled_doublesGameDiff_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesWeightedGameDiff_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesWinPercentage_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesOpponentUtr_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesLongestWinStreak_train_samples)

serveSpeed_train_labels = []
scaled_serveSpeed_train_labels = []

##########################################################################################################################################
'''
singlesUtr_valid_samples = []
doublesUtr_valid_samples = []
servePercentage_valid_samples = []

singlesGameDiff_valid_samples = []
singlesWeightedGameDiff_valid_samples = []
singlesWinPercentage_valid_samples = []
singlesOpponentUtr_valid_samples = []
singlesLongestWinStreak_valid_samples = []

valid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]
scaled_valid_sets["singles"] = [scaled_singlesGameDiff_valid_samples, scaled_singlesWeightedGameDiff_valid_samples, scaled_singlesWinPercentage_valid_samples, scaled_singlesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]


doublesGameDiff_valid_samples = []
doublesWeightedGameDiff_valid_samples = []
doublesWinPercentage_valid_samples = []
doublesOpponentUtr_valid_samples = []
doublesLongestWinStreak_valid_samples = []

valid_sets["doubles"] = [doublesGameDiff_valid_samples, doublesWeightedGameDiff_valid_samples, doublesWinPercentage_valid_samples, doublesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]
scaled_valid_sets["doubles"] = [scaled_doublesGameDiff_valid_samples, scaled_doublesWeightedGameDiff_valid_samples, scaled_doublesWinPercentage_valid_samples, scaled_doublesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]
'''


# serveSpeed_valid_labels = []

#######################################################################################################################################



'\nsinglesUtr_valid_samples = []\ndoublesUtr_valid_samples = []\nservePercentage_valid_samples = []\n\nsinglesGameDiff_valid_samples = []\nsinglesWeightedGameDiff_valid_samples = []\nsinglesWinPercentage_valid_samples = []\nsinglesOpponentUtr_valid_samples = []\nsinglesLongestWinStreak_valid_samples = []\n\nvalid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]\nscaled_valid_sets["singles"] = [scaled_singlesGameDiff_valid_samples, scaled_singlesWeightedGameDiff_valid_samples, scaled_singlesWinPercentage_valid_samples, scaled_singlesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]\n\n\ndoublesGameDiff_valid_samples = []\ndoublesWeightedGameDiff_valid_samples = []\ndoublesWinPercentage_valid_samples = []\ndoublesOpponentUtr_valid_samples = []\ndoublesLongestWinStreak_valid_samples = []\n\nvalid_sets["doubles"] = [doub

In [6]:
adv_testing_sets = dict()
scaled_adv_testing_sets = dict()


singlesUtr_test_samples = []
doublesUtr_test_samples = []
servePercentage_test_samples = []

scaled_singlesUtr_test_samples = []
scaled_doublesUtr_test_samples = []
scaled_servePercentage_test_samples = []

basic_testing_sets = []
basic_testing_sets.append(singlesUtr_test_samples)
basic_testing_sets.append(doublesUtr_test_samples) 
basic_testing_sets.append(servePercentage_test_samples)

scaled_basic_testing_sets = []
scaled_basic_testing_sets.append(scaled_singlesUtr_test_samples) 
scaled_basic_testing_sets.append(scaled_doublesUtr_test_samples) 
scaled_basic_testing_sets.append(scaled_servePercentage_test_samples)

singlesGameDiff_test_samples = []
singlesWeightedGameDiff_test_samples = []
singlesWinPercentage_test_samples = []
singlesOpponentUtr_test_samples = []
singlesLongestWinStreak_test_samples = []

scaled_singlesGameDiff_test_samples = []
scaled_singlesWeightedGameDiff_test_samples = []
scaled_singlesWinPercentage_test_samples = []
scaled_singlesOpponentUtr_test_samples = []
scaled_singlesLongestWinStreak_test_samples = []

adv_testing_sets["singles"] = [] 
adv_testing_sets["singles"].append(singlesGameDiff_test_samples) 
adv_testing_sets["singles"].append(singlesWeightedGameDiff_test_samples) 
adv_testing_sets["singles"].append(singlesWinPercentage_test_samples) 
adv_testing_sets["singles"].append(singlesOpponentUtr_test_samples) 
adv_testing_sets["singles"].append(singlesLongestWinStreak_test_samples)

scaled_adv_testing_sets["singles"] = [] 
scaled_adv_testing_sets["singles"].append(scaled_singlesGameDiff_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesWeightedGameDiff_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesWinPercentage_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesOpponentUtr_test_samples) 
scaled_adv_testing_sets["singles"].append(scaled_singlesLongestWinStreak_test_samples)


# print(scaled_adv_testing_sets)

'''
for i in range(5):   # Adding scaled values to all advanced stat testing sets
    # list_ = adv_testing_sets[match_type][i]
    scaled_list = scaled_adv_testing_sets['singles'][i]
    scaled_adv_testing_sets['singles'][i].extend([0, i])
    # print(f"adv_testing_set scaled list: {type(scaled_adv_testing_sets[dict_string][i])}")

print(scaled_adv_testing_sets)

print(type(scaled_singlesGameDiff_test_samples))
'''

# print(singles_test_lists)

doublesGameDiff_test_samples = []
doublesWeightedGameDiff_test_samples = []
doublesWinPercentage_test_samples = []
doublesOpponentUtr_test_samples = []
doublesLongestWinStreak_test_samples = []

scaled_doublesGameDiff_test_samples = []
scaled_doublesWeightedGameDiff_test_samples = []
scaled_doublesWinPercentage_test_samples = []
scaled_doublesOpponentUtr_test_samples = []
scaled_doublesLongestWinStreak_test_samples = []

'''
adv_testing_sets["doubles"] = [doublesGameDiff_test_samples, doublesWeightedGameDiff_test_samples, doublesWinPercentage_test_samples, doublesOpponentUtr_test_samples, doublesLongestWinStreak_test_samples]
scaled_adv_testing_sets["doubles"] = [scaled_doublesGameDiff_test_samples, scaled_doublesWeightedGameDiff_test_samples, scaled_doublesWinPercentage_test_samples, scaled_doublesOpponentUtr_test_samples, scaled_doublesLongestWinStreak_test_samples]
'''

adv_testing_sets["doubles"] = [] 
adv_testing_sets["doubles"].append(doublesGameDiff_test_samples) 
adv_testing_sets["doubles"].append(doublesWeightedGameDiff_test_samples) 
adv_testing_sets["doubles"].append(doublesWinPercentage_test_samples) 
adv_testing_sets["doubles"].append(doublesOpponentUtr_test_samples) 
adv_testing_sets["doubles"].append(doublesLongestWinStreak_test_samples)
                                    
scaled_adv_testing_sets["doubles"] = [] 
scaled_adv_testing_sets["doubles"].append(scaled_doublesGameDiff_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesWeightedGameDiff_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesWinPercentage_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesOpponentUtr_test_samples) 
scaled_adv_testing_sets["doubles"].append(scaled_doublesLongestWinStreak_test_samples)

serveSpeed_test_labels = []
scaled_serveSpeed_test_labels = []

In [7]:
print(len(data))

76


### Getting Training and Varification sets data from data dictionary

In [8]:
# Need to do: get all data from data dict into training_labels and training_sameples
# Each stat can go in its individual list, I will just have to be careful that every individual number (Stat) for each player is the same 
# index in each list. For example, Sinner's serve percentage will have the same index as his singles UTR, doubles UTR, etc. 

In [9]:
# PS - only lists with a string ("no data?") are the serve speed lists!

name_list = list(data)

match_types = ["singles", "doubles"]

# Training Set
for i in range(70):
    
    training_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_train_labels.append(player[0])   # appending serve speed to label list

    for i in range(3):
        # print(player[i+1])
        if player[i+1] != 0:
            basic_training_sets[i].append(player[i+1])  # appending serve percentage, singles utr, and doubles utr to training set lists
        else:
            basic_training_sets[i].append(None)    # If singles or doubles UTR, or serve percentage equal 0, append None to the training set
    '''   
    servePercentage_train_samples.append(player[1])
    singlesUtr_train_samples.append(player[2])
    doublesUtr_train_samples.append(player[3])
    '''

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        # print(match_type_data)
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in adv_training_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): # Append data from data dict to appropriate training set lists
                adv_training_sets[dict_string][i].append(match_type_data[i])  


###### Testing Set #######################################################################################################################

print(f"Remaining name_list: {name_list}")

for name in name_list:
    print(name)
    # testing_list = []
    # name = random.choice(name_list)
    # name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_test_labels.append(player[0])   # appending serve speed to label list

    for i in range(3):
        # print(player[i+1])
        if player[i+1] != 0:
            basic_testing_sets[i].append(player[i+1])  # appending serve percentage, singles utr, and doubles utr to training set lists
        else:
            basic_testing_sets[i].append(None)    # If singles or doubles UTR, or serve percentage equal 0, append None to the training set
    '''   
    servePercentage_train_samples.append(player[1])
    singlesUtr_train_samples.append(player[2])
    doublesUtr_train_samples.append(player[3])
    '''

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        print(match_type_data)
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in adv_testing_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): # Append data from data dict to appropriate training set lists
                adv_testing_sets[dict_string][i].append(match_type_data[i]) 
        
print(f"Final name list: {name_list}")
# Validation Set ##################################################################################################################
'''
for i in range(10):
    valid_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_valid_labels.append(player[0])

    servePercentage_valid_samples.append(player[1])
    singlesUtr_valid_samples.append(player[2])
    doublesUtr_valid_samples.append(player[3])

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in valid_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): 
                valid_sets[dict_string][i].append(match_type_data[i])              
            
'''
    


print(f"Players left for test set: {len(name_list)}")


Remaining name_list: ['Novak Djokovic', 'Stefanos Tsitsipas', 'Alexandre Vita', 'Gaston Deferrari', 'Hendrik Te Grotenhuis', 'Lisa Kesler']
Novak Djokovic
[15.33, 212.0, 130.79, 82.22, 8]
[13.82, 5.0, 1.71, 100.0, 2]
Stefanos Tsitsipas
[15.33, 153.0, 119.89, 67.16, 10]
[13.78, -40.0, -61.12, 28.0, 3]
Alexandre Vita
[6.9, 13.0, 9.8, 60.0, 3]
None
Gaston Deferrari
[7.5, 5.0, 6.0, 50.0, 1]
[7.0, 5.0, 4.45, 100.0, 1]
Hendrik Te Grotenhuis
[7.36, 32.0, 31.6, 71.43, 6]
[6.88, -8.0, -23.3, 37.5, 2]
Lisa Kesler
None
[4.43, 132.0, 141.75, 69.77, 9]
Final name list: ['Novak Djokovic', 'Stefanos Tsitsipas', 'Alexandre Vita', 'Gaston Deferrari', 'Hendrik Te Grotenhuis', 'Lisa Kesler']
Players left for test set: 6


In [10]:
print(name_list)

['Novak Djokovic', 'Stefanos Tsitsipas', 'Alexandre Vita', 'Gaston Deferrari', 'Hendrik Te Grotenhuis', 'Lisa Kesler']


In [11]:
print(f"elements in data: {len(data)}")
print(f"Elements left to use for test set: {len(name_list)}")

elements in data: 76
Elements left to use for test set: 6


In [12]:
scaler = MinMaxScaler(feature_range=(0,1))

print(scaled_basic_training_sets)
def scaler_func(list_, scaled_list):

    # list_ = np.array(list_)
    '''
    for elem in list_:
        if elem != 'No data?' and elem != None:
            elem = float(elem)
        else:
            elem = np.array([0])
    '''
    
    # list_ = [float(x) if x != 'No data?' and x != None else np.array([0]) for x in list_]

    # print(list_)
    numeric_data = [x if isinstance(x, (int, float)) else 0 for x in list_]  # creates a new list numeric_data that contains all values that are intagers and floats
    numeric_data = np.array(numeric_data).reshape(-1, 1)  # Converts all numbers in training set to numpy. Training sets with None/string values can't be converted to numpy (I think)
    scaled_numeric_data = scaler.fit_transform((numeric_data))
    
    # scaled_list = scaled_basic_training_sets[i]
    numeric_index = 0

    '''
    for item in list_:     # If item in basic_training_sets list is string or None, append that. If item is int or float, append next scaled_numeric_data value 
        if isinstance(item, (int, float)):
            scaled_list.append(scaled_numeric_data[numeric_index])
            numeric_index += 1
        else:
            scaled_list.append(np.array([0]))
    '''

    # print(f"scaled_list = {scaled_list}")

    # scaled_list.clear()

    # print(scaled_list)
    '''
    if scaled_list != None:
        scaled_list[:] = scaled_numeric_data.reshape(70,1) # manually changed the shape
    else:
        scaled_list = scaled_numeric_data.flatten()
    '''

    # scaled_list = [0 if x != 'No data?' or x != None else x for x in scaled_list]
    
    # print(f"scaled_numeric_data = {scaled_numeric_data}")

    return scaled_numeric_data
            



# serveSpeed_train_labels = np.array(serveSpeed_train_labels)

print(f"pre-scaled serve speed training labels: {type(scaled_serveSpeed_train_labels)}")

scaled_serveSpeed_train_labels = scaler_func(serveSpeed_train_labels, scaled_serveSpeed_train_labels)

scaled_singlesUtr_train_samples = scaler_func(singlesUtr_train_samples, scaled_singlesUtr_train_samples)

scaled_doublesUtr_train_samples = scaler_func(doublesUtr_train_samples, scaled_doublesUtr_train_samples)

scaled_servePercentage_train_samples = scaler_func(servePercentage_train_samples, scaled_servePercentage_train_samples)

scaled_singlesGameDiff_train_samples = scaler_func(singlesGameDiff_train_samples, scaled_singlesGameDiff_train_samples)

scaled_singlesWeightedGameDiff_train_samples = scaler_func(singlesWeightedGameDiff_train_samples, scaled_singlesWeightedGameDiff_train_samples)

scaled_singlesWinPercentage_train_samples = scaler_func(singlesWinPercentage_train_samples, scaled_singlesWinPercentage_train_samples)

scaled_singlesOpponentUtr_train_samples = scaler_func(singlesOpponentUtr_train_samples, scaled_singlesOpponentUtr_train_samples)

scaled_singlesLongestWinStreak_train_samples = scaler_func(singlesLongestWinStreak_train_samples, scaled_singlesLongestWinStreak_train_samples)

print(f"Scaled serve speed training lables: {scaled_serveSpeed_train_labels}")







'''
for i in range(3):   # Adding scaled values to all basic training sets
    list_ = basic_training_sets[i]
    scaled_list = scaled_basic_training_sets[i]

    print(f"pre-scaled scaled_basic_training_sets[0] = {type(scaled_basic_training_sets[0])}")

    scaled_basic_training_sets[i] = scaler_func(basic_training_sets[i], scaled_basic_training_sets[i])

    # scaled_basic_training_sets[i] = np.array(scaled_basic_training_sets[i]).reshape(-1, 1)

    print(f"basic_training_set scaled list type: {type(scaled_basic_training_sets[i])}")
    print(f"basic_training_set scaled list: {scaled_basic_training_sets[i]}")


print(f"scaled_singlesUtr_train_samples type = {type(scaled_singlesUtr_train_samples)}")
print(f"scaled_singlesUtr_train_samples = {scaled_singlesUtr_train_samples}")


for match_type in match_types:  
    dict_string = f"{match_type}"
    dict_string = 'singles'    # CHANGE LATER 
    # match_type_data = player[4][dict_string]

    for i in range(5):   # Adding scaled values to all advanced stat training sets
        list_ = adv_training_sets[match_type][i]
        scaled_list = scaled_adv_training_sets[dict_string][i]
        scaled_adv_training_sets[dict_string][i].extend(scaler_func(list_, scaled_list))
        # scaled_adv_training_sets[dict_string][i] = scaler.fit_transform((scaled_adv_training_sets[dict_string][i]).reshape(-1,1))
        print(f"adv_training_set scaled list: {type(scaled_adv_training_sets[dict_string][i])}")

      
'''


[[], [], []]
pre-scaled serve speed training labels: <class 'list'>
Scaled serve speed training lables: [[0.86040444]
 [0.        ]
 [0.87837936]
 [0.87837936]
 [0.86040444]
 [0.49648474]
 [0.89642676]
 [0.4421251 ]
 [0.90541422]
 [0.91896789]
 [0.86489817]
 [0.52547655]
 [0.84235703]
 [0.40950931]
 [0.8693919 ]
 [0.456621  ]
 [0.88287309]
 [0.        ]
 [0.50735667]
 [0.44937305]
 [0.88287309]
 [0.89193303]
 [0.84235703]
 [0.86489817]
 [0.8378633 ]
 [0.94600275]
 [0.89193303]
 [0.8378633 ]
 [0.87837936]
 [0.43125317]
 [0.43850112]
 [0.41313329]
 [0.46386896]
 [0.        ]
 [0.89193303]
 [0.43125317]
 [0.        ]
 [0.49648474]
 [0.46024498]
 [0.8874393 ]
 [0.88287309]
 [1.        ]
 [0.83336957]
 [0.3805175 ]
 [0.        ]
 [0.8874393 ]
 [0.86489817]
 [0.        ]
 [0.8693919 ]
 [0.91447416]
 [0.90092049]
 [0.90990795]
 [0.47474089]
 [0.87388563]
 [0.        ]
 [0.91896789]
 [0.5182286 ]
 [0.        ]
 [0.31166196]
 [0.        ]
 [0.86489817]
 [0.54359643]
 [0.82887584]
 [0.86040444]


'\nfor i in range(3):   # Adding scaled values to all basic training sets\n    list_ = basic_training_sets[i]\n    scaled_list = scaled_basic_training_sets[i]\n\n    print(f"pre-scaled scaled_basic_training_sets[0] = {type(scaled_basic_training_sets[0])}")\n\n    scaled_basic_training_sets[i] = scaler_func(basic_training_sets[i], scaled_basic_training_sets[i])\n\n    # scaled_basic_training_sets[i] = np.array(scaled_basic_training_sets[i]).reshape(-1, 1)\n\n    print(f"basic_training_set scaled list type: {type(scaled_basic_training_sets[i])}")\n    print(f"basic_training_set scaled list: {scaled_basic_training_sets[i]}")\n\n\nprint(f"scaled_singlesUtr_train_samples type = {type(scaled_singlesUtr_train_samples)}")\nprint(f"scaled_singlesUtr_train_samples = {scaled_singlesUtr_train_samples}")\n\n\nfor match_type in match_types:  \n    dict_string = f"{match_type}"\n    dict_string = \'singles\'    # CHANGE LATER \n    # match_type_data = player[4][dict_string]\n\n    for i in range(5): 

In [13]:
print(f"pre-scaled serve speed testing labels: {type(scaled_serveSpeed_test_labels)}")

scaled_serveSpeed_test_labels = scaler_func(serveSpeed_test_labels, scaled_serveSpeed_test_labels)

scaled_singlesUtr_test_samples = scaler_func(singlesUtr_test_samples, scaled_singlesUtr_test_samples)

scaled_doublesUtr_test_samples = scaler_func(doublesUtr_test_samples, scaled_doublesUtr_test_samples)

scaled_servePercentage_test_samples = scaler_func(servePercentage_test_samples, scaled_servePercentage_test_samples)

scaled_singlesGameDiff_test_samples = scaler_func(singlesGameDiff_test_samples, scaled_singlesGameDiff_test_samples)

scaled_singlesWeightedGameDiff_test_samples = scaler_func(singlesWeightedGameDiff_test_samples, scaled_singlesWeightedGameDiff_test_samples)

scaled_singlesWinPercentage_test_samples = scaler_func(singlesWinPercentage_test_samples, scaled_singlesWinPercentage_test_samples)

scaled_singlesOpponentUtr_test_samples = scaler_func(singlesOpponentUtr_test_samples, scaled_singlesOpponentUtr_test_samples)

scaled_singlesLongestWinStreak_test_samples = scaler_func(singlesLongestWinStreak_test_samples, scaled_singlesLongestWinStreak_test_samples)

print(f"Scaled serve speed testing lables: {scaled_serveSpeed_test_labels}")

pre-scaled serve speed testing labels: <class 'list'>
Scaled serve speed testing lables: [[0.92860792]
 [1.        ]
 [0.19157088]
 [0.20434227]
 [0.37037037]
 [0.        ]]


In [14]:
'''

print(scaled_serveSpeed_train_labels.shape)
print(scaled_singlesUtr_train_samples.shape)
print(scaled_doublesUtr_train_samples.shape)
print(scaled_servePercentage_train_samples.shape)
print(scaled_singlesGameDiff_train_samples.shape)
print(scaled_singlesWeightedGameDiff_train_samples.shape)
print(scaled_singlesWinPercentage_train_samples.shape)
print(scaled_singlesOpponentUtr_train_samples.shape)
print(scaled_singlesLongestWinStreak_train_samples.shape)



print(len(scaled_singlesUtr_train_samples))
print(len(scaled_doublesUtr_train_samples))
print(len(scaled_servePercentage_train_samples))
print(len(scaled_singlesGameDiff_train_samples))
print(len(scaled_singlesWeightedGameDiff_train_samples))
print(len(scaled_singlesWinPercentage_train_samples))
print(len(scaled_singlesOpponentUtr_train_samples))
print(len(scaled_singlesLongestWinStreak_train_samples))

'''

print(type(scaled_singlesUtr_train_samples))
print(type(scaled_doublesUtr_train_samples))
print(type(scaled_servePercentage_train_samples))
print(type(scaled_singlesGameDiff_train_samples))
print(type(scaled_singlesWeightedGameDiff_train_samples))
print(type(scaled_singlesWinPercentage_train_samples))
print(type(scaled_singlesOpponentUtr_train_samples))
print(type(scaled_singlesLongestWinStreak_train_samples))
print(type(scaled_serveSpeed_train_labels))


'''

print(scaled_singlesUtr_train_samples)
print(scaled_doublesUtr_train_samples)
print(scaled_servePercentage_train_samples)
print(scaled_singlesGameDiff_train_samples)
print(scaled_singlesWeightedGameDiff_train_samples)
print(scaled_singlesWinPercentage_train_samples)
print(scaled_singlesOpponentUtr_train_samples)
print(scaled_singlesLongestWinStreak_train_samples)
print(scaled_serveSpeed_train_labels)
'''

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


'\n\nprint(scaled_singlesUtr_train_samples)\nprint(scaled_doublesUtr_train_samples)\nprint(scaled_servePercentage_train_samples)\nprint(scaled_singlesGameDiff_train_samples)\nprint(scaled_singlesWeightedGameDiff_train_samples)\nprint(scaled_singlesWinPercentage_train_samples)\nprint(scaled_singlesOpponentUtr_train_samples)\nprint(scaled_singlesLongestWinStreak_train_samples)\nprint(scaled_serveSpeed_train_labels)\n'

In [15]:
'''

print(scaled_serveSpeed_test_labels.shape)
print(scaled_singlesUtr_test_samples.shape)
print(scaled_doublesUtr_test_samples.shape)
print(scaled_servePercentage_test_samples.shape)
print(scaled_singlesGameDiff_test_samples.shape)
print(scaled_singlesWeightedGameDiff_test_samples.shape)
print(scaled_singlesWinPercentage_test_samples.shape)
print(scaled_singlesOpponentUtr_test_samples.shape)
print(scaled_singlesLongestWinStreak_test_samples.shape)



print(len(scaled_singlesUtr_test_samples))
print(len(scaled_doublesUtr_test_samples))
print(len(scaled_servePercentage_test_samples))
print(len(scaled_singlesGameDiff_test_samples))
print(len(scaled_singlesWeightedGameDiff_test_samples))
print(len(scaled_singlesWinPercentage_test_samples))
print(len(scaled_singlesOpponentUtr_test_samples))
print(len(scaled_singlesLongestWinStreak_test_samples))

'''

print(type(scaled_singlesUtr_test_samples))
print(type(scaled_doublesUtr_test_samples))
print(type(scaled_servePercentage_test_samples))
print(type(scaled_singlesGameDiff_test_samples))
print(type(scaled_singlesWeightedGameDiff_test_samples))
print(type(scaled_singlesWinPercentage_test_samples))
print(type(scaled_singlesOpponentUtr_test_samples))
print(type(scaled_singlesLongestWinStreak_test_samples))
print(type(scaled_serveSpeed_test_labels))


'''

print(scaled_singlesUtr_test_samples)
print(scaled_doublesUtr_test_samples)
print(scaled_servePercentage_test_samples)
print(scaled_singlesGameDiff_test_samples)
print(scaled_singlesWeightedGameDiff_test_samples)
print(scaled_singlesWinPercentage_test_samples)
print(scaled_singlesOpponentUtr_test_samples)
print(scaled_singlesLongestWinStreak_test_samples)
print(scaled_serveSpeed_test_labels)
'''

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


'\n\nprint(scaled_singlesUtr_test_samples)\nprint(scaled_doublesUtr_test_samples)\nprint(scaled_servePercentage_test_samples)\nprint(scaled_singlesGameDiff_test_samples)\nprint(scaled_singlesWeightedGameDiff_test_samples)\nprint(scaled_singlesWinPercentage_test_samples)\nprint(scaled_singlesOpponentUtr_test_samples)\nprint(scaled_singlesLongestWinStreak_test_samples)\nprint(scaled_serveSpeed_test_labels)\n'

In [16]:
print(scaled_serveSpeed_test_labels)

[[0.92860792]
 [1.        ]
 [0.19157088]
 [0.20434227]
 [0.37037037]
 [0.        ]]


### Making the Model

In [17]:
end_sets = np.column_stack((scaled_singlesUtr_train_samples, scaled_doublesUtr_train_samples, scaled_servePercentage_train_samples,
                           scaled_singlesGameDiff_train_samples, scaled_singlesWeightedGameDiff_train_samples, 
                            scaled_singlesWinPercentage_train_samples, scaled_singlesOpponentUtr_train_samples, 
                            scaled_singlesLongestWinStreak_train_samples))

print(end_sets)

[[0.65301205 0.95227442 0.91450777 0.96260477 0.57894737 0.41900245
  0.70551351 0.17241379]
 [0.49156627 0.91573453 0.96696891 0.96647324 0.43157895 0.47317919
  0.66410811 0.13793103]
 [0.52289157 0.950783   0.96632124 0.98903933 0.42556391 0.44902478
  0.67837838 0.31034483]
 [0.62650602 0.93512304 0.93782383 0.97807866 0.41654135 0.43891277
  0.55740541 0.13793103]
 [0.63614458 0.94556301 0.94106218 0.98903933 0.57142857 0.62059676
  0.72551351 0.31034483]
 [0.3253012  0.47800149 0.57772021 0.62862669 0.32631579 0.41961139
  0.59459459 0.10344828]
 [0.45783133 0.93437733 0.97215026 0.98646035 0.43308271 0.49094902
  0.61189189 0.20689655]
 [0.60240964 0.12602535 0.34715026 0.31592521 0.34586466 0.43455797
  0.43243243 0.06896552]
 [0.69156627 0.91498881 0.93523316 0.97936815 0.31879699 0.37842526
  0.52248649 0.17241379]
 [0.77108434 0.98210291 0.96243523 0.99484204 0.8        0.74922961
  0.82962162 0.4137931 ]
 [0.61686747 0.9448173  0.97020725 0.98774984 0.47218045 0.49366154
  

In [18]:
test_sets = np.column_stack((scaled_singlesUtr_test_samples, scaled_doublesUtr_test_samples, scaled_servePercentage_test_samples,
                           scaled_singlesGameDiff_test_samples, scaled_singlesWeightedGameDiff_test_samples, 
                            scaled_singlesWinPercentage_test_samples, scaled_singlesOpponentUtr_test_samples, 
                            scaled_singlesLongestWinStreak_test_samples))

print(test_sets)

[[0.63684211 1.         1.         1.         1.         1.
  1.         0.8       ]
 [0.44210526 0.9460371  0.97315898 1.         0.72169811 0.9166603
  0.81683289 1.        ]
 [0.57894737 0.25463744 0.         0.45009785 0.06132075 0.07492928
  0.72974945 0.3       ]
 [1.         0.29005059 0.52305575 0.48923679 0.02358491 0.04587507
  0.60812454 0.1       ]
 [0.         0.29342327 0.51961459 0.48010437 0.1509434  0.24160869
  0.86876672 0.6       ]
 [0.65789474 0.         0.33172746 0.         0.         0.
  0.         0.        ]]


In [19]:

model = Sequential([
    Input(shape=(8,1)),
    Dense(64, activation='relu'),
    Dropout(0.1),
    BatchNormalization(),
    
    Dense(32, activation='relu'),
    Dropout(0.1),
    BatchNormalization(),
    
    Dense(16, activation='relu'),
    
    Dense(1)
])


### Training the Model

In [20]:
model.compile(Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

In [21]:
model.fit(end_sets, scaled_serveSpeed_train_labels, validation_split=0.1, batch_size=10, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
7/7 - 1s - 77ms/step - loss: 1.8797 - mae: 0.9886 - val_loss: 0.3988 - val_mae: 0.6001
Epoch 2/1000
7/7 - 0s - 3ms/step - loss: 1.7620 - mae: 0.9531 - val_loss: 0.4143 - val_mae: 0.6131
Epoch 3/1000
7/7 - 0s - 3ms/step - loss: 1.5777 - mae: 0.8929 - val_loss: 0.4312 - val_mae: 0.6258
Epoch 4/1000
7/7 - 0s - 3ms/step - loss: 1.4513 - mae: 0.8704 - val_loss: 0.4396 - val_mae: 0.6321
Epoch 5/1000
7/7 - 0s - 3ms/step - loss: 1.3996 - mae: 0.8783 - val_loss: 0.4535 - val_mae: 0.6421
Epoch 6/1000
7/7 - 0s - 3ms/step - loss: 1.1803 - mae: 0.7974 - val_loss: 0.4650 - val_mae: 0.6501
Epoch 7/1000
7/7 - 0s - 3ms/step - loss: 0.9702 - mae: 0.7276 - val_loss: 0.4730 - val_mae: 0.6553
Epoch 8/1000
7/7 - 0s - 3ms/step - loss: 0.9272 - mae: 0.7318 - val_loss: 0.4756 - val_mae: 0.6569
Epoch 9/1000
7/7 - 0s - 3ms/step - loss: 0.9495 - mae: 0.7305 - val_loss: 0.4775 - val_mae: 0.6581
Epoch 10/1000
7/7 - 0s - 3ms/step - loss: 0.8004 - mae: 0.6759 - val_loss: 0.4784 - val_mae: 0.6585
Epoch 11

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8, 64)          │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8, 16)          │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8, 1)           │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,029 (35.27 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 5,892 (23.02 KB)

### Preprocessing Test Data

In [23]:
serve_speed_predictions = model.predict(test_sets, batch_size=1, verbose=0)

In [24]:
# model(model, to_file='model.png', show_shapes=True)

In [26]:
print(f"confidence\n")
for predictions in serve_speed_predictions:
    for i in range(6):
        label = scaled_serveSpeed_test_labels[i]
        name = name_list[i]
        prediction = predictions[i]
        print(f"{name}: Guess = {prediction}, Speed = {label}")
    print()

confidence

Novak Djokovic: Guess = [0.69800633], Speed = [0.92860792]
Stefanos Tsitsipas: Guess = [0.68476087], Speed = [1.]
Alexandre Vita: Guess = [0.68476087], Speed = [0.19157088]
Gaston Deferrari: Guess = [0.68476087], Speed = [0.20434227]
Hendrik Te Grotenhuis: Guess = [0.68476087], Speed = [0.37037037]
Lisa Kesler: Guess = [0.68476087], Speed = [0.]

Novak Djokovic: Guess = [0.54098207], Speed = [0.92860792]
Stefanos Tsitsipas: Guess = [0.6218219], Speed = [1.]
Alexandre Vita: Guess = [0.6534552], Speed = [0.19157088]
Gaston Deferrari: Guess = [0.68476087], Speed = [0.20434227]
Hendrik Te Grotenhuis: Guess = [0.6820024], Speed = [0.37037037]
Lisa Kesler: Guess = [0.5875587], Speed = [0.]

Novak Djokovic: Guess = [0.68295765], Speed = [0.92860792]
Stefanos Tsitsipas: Guess = [0.51830214], Speed = [1.]
Alexandre Vita: Guess = [0.46291676], Speed = [0.19157088]
Gaston Deferrari: Guess = [0.5354019], Speed = [0.20434227]
Hendrik Te Grotenhuis: Guess = [0.47398785], Speed = [0.37037